In [64]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm

pd.set_option('display.max_columns', None)

In [52]:
ess = pd.read_csv('ESS10.csv')

/tmp/ipykernel_445065/1309307712.py:1: DtypeWarning: Columns (159,165,167,578) have mixed types. Specify dtype option on import or set low_memory=False.
  ess = pd.read_csv('ESS10.csv')


In [53]:
print(ess.columns[159],ess.columns[165],ess.columns[167],ess.columns[578])

cntbrthd fbrncntc mbrncntc kinwe


In [54]:
ess.dtypes.value_counts()

int64      385
float64    177
object      24
dtype: int64

In [55]:
# have to treat float columns:
# most of them are float because they contain NaNs (country specific stuff)
# BUT some not

# there is pandas type 'Int64' 
# which:    can treat nans as 'integers'
#           won't treat floats
#           can use max() method even with NaNs

# want to convert all float to Int64 if possible

for col in tqdm(ess.select_dtypes(np.number).columns):
    # use the nice properties of Int64 with a try-except
    try:
        ess[col] = ess[col].astype('Int64')
    except:
        print(col, 'is a true float col !')


  1%|          | 3/562 [00:00<00:43, 12.78it/s]

edition is a true float col !
dweight is a true float col !


  2%|▏         | 10/562 [00:00<00:21, 26.11it/s]

pspwght is a true float col !
pweight is a true float col !
anweight is a true float col !
prob is a true float col !


100%|██████████| 562/562 [00:07<00:00, 73.28it/s] 


In [56]:
ess.dtypes.value_counts()

Int64      556
object      24
float64      6
dtype: int64

In [57]:
# test the nan filling with a random int-column
rand_column = ess.select_dtypes(int).columns[random.randint(0,len(ess.select_dtypes(int).columns)-1)]
ess[rand_column].value_counts(dropna=False)

<NA>    31828
12        517
3         170
2         121
8888      105
14         88
11         69
16         62
23         61
22         59
15         32
4          29
17         29
18         28
20         28
13         27
1          19
5          16
6          14
8          11
7777        9
9           9
21          8
19          7
7           3
10          1
5555        1
Name: edlvfdch, dtype: Int64

In [58]:
# replace nan filler with actual nans
for col in tqdm(ess.select_dtypes(int).columns):
    # fillers are either 6,7,8,9 times the number of digits of max of col OR the filler is max itself
    length = len(str(int(ess[col].max())))
    replace = [int(str(i)*length) for i in range (6,10)]
    ess[col] = ess[col].replace(replace, np.nan)

100%|██████████| 556/556 [00:00<00:00, 974.71it/s] 


In [59]:
ess[rand_column].value_counts(dropna=False)

<NA>    31942
12        517
3         170
2         121
14         88
11         69
16         62
23         61
22         59
15         32
17         29
4          29
18         28
20         28
13         27
1          19
5          16
6          14
8          11
9           9
21          8
19          7
7           3
10          1
5555        1
Name: edlvfdch, dtype: Int64

In [61]:
obj_cols = ess.select_dtypes(object).columns   
# 'name', 'proddate', 'cntry', 'cntbrthd', 'lnghom1', 'lnghom2','fbrncntc', 'mbrncntc', 'region', 'inwds', 'ainws', 'ainwe', 'binwe','cinwe', 'dinwe', 'finwe', 'ginwe', 'hinwe', 'iinwe', 'kinwe', 'vinwe','inwde', 'jinws', 'jinwe'         

# we drop all but region & cntry             
ess = ess.drop([col for col in obj_cols if col not in ['cntry', 'region']] + ['psu','essround','edition','stratum','idno'], axis=1)
                                                                # constants, ids ... should get eliminated later anyway, 
                                                                #                    because of variance/multicoll
ess

,cntry,dweight,pspwght,pweight,anweight,prob,nwspol,netusoft,netustm,ppltrst,pplfair,pplhlp,polintr,psppsgva,actrolga,psppipla,cptppola,trstprl,trstlgl,trstplc,trstplt,trstprt,trstep,trstun,trstsci,vote,prtvtebg,prtvthch,prtvtbhr,prtvtecz,prtvthee,prtvtefi,prtvtefr,prtvtdgr,prtvtghu,prtvtdis,prtvtdit,prtvclt1,prtvclt2,prtvclt3,prtvtame,prtvthnl,prtvtmk,prtvtbno,prtvtdpt,prtvtfsi,prtvtesk,contplt,donprty,badge,sgnptit,pbldmna,bctprd,pstplonl,volunfp,clsprty,prtclebg,prtclhch,prtclbhr,prtclecz,prtclhee,prtclffi,prtclffr,prtcldgr,prtclhhu,prtcldis,prtcleit,prtclclt,prtclame,prtclgnl,prtclmk,prtclbno,prtclfpt,prtclfsi,prtclesk,prtdgcl,lrscale,stflife,stfeco,stfgov,stfdem,stfedu,stfhlth,gincdif,freehms,hmsfmlsh,hmsacld,euftf,lrnobed,loylead,imsmetn,imdfetn,impcntr,imbgeco,imueclt,imwbcnt,happy,sclmeet,inprdsc,sclact,...,c19whacc,mansupp,manhlp,manwrkpl,manspeak,manscrn,manphone,mancom,teamfeel,wrkextra,colprop,colhlp,colspeak,colscrn,colphone,colcom,c19spwrk,c19mcwrk,mcwrkhom,ipcrtiv,imprich,ipeqopt,ipshabt,impsafe,impdiff,ipfrule,ipudrst,ipmodst,ipgdtim,impfree,iphlppl,ipsuces,ipstrgv,ipadvnt,ipbhprp,iprspot,iplylfr,impenv,imptrad,impfun,testii1,testii2,testii3,testii4,testii5,testii6,testii7,testii8,testii9,secgrdec,scidecpb,admc19,panpriph,panmonpb,govpriph,govmonpb,panfolru,panclobo,panresmo,gvhanc19,gvjobc19,gveldc19,gvfamc19,hscopc19,gvbalc19,gvimpc19,gvconc19,respc19,reshhc19,hapljc19,hapirc19,hapwrc19,hapfuc19,hapfoc19,hapnoc19,hapnwc19,hapnpc19,haprec19,hapdkc19,hapnac19,icvacc19,getavc19,getnvc19,vdcond,vdovexre,vdtype,vdtpsvre,vdtpitre,vdtpscre,vdtpaure,vdtpvire,vdtpoire,vdtpntre,vdtpapre,vdtprere,vdtpdkre,vdtpnare,inwtm,mode,domain
0,BG,1.939384,1.290706,0.217716,0.281008,0.000314,80,1,<NA>,5,5,1,4,1,2,2,2,3,2,3,3,3,4,4,6,2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2,2,2,2,2,2,2,2,2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,8,3,4,4,3,4,4,3,3,3,2,3,3,3,3,3,2,2,2,6,6,2,3,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2,2,2,5,3,1,2,4,4,2,1,3,5,3,3,2,5,4,3,1,5,3,3,3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2,1,2,<NA>,<NA>,1,2,8,3,8,7,1,8,8,8,5,8,1,3,4,0,0,0,0,0,0,0,1,0,0,0,<NA>,3,<NA>,1,9,<NA>,0,0,0,0,0,0,0,1,0,0,0,53,1,<NA>
1,BG,1.651595,1.430878,0.217716,0.311526,0.000368,63,5,180,0,6,2,1,4,4,4,4,5,8,9,6,7,8,8,10,1,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,2,2,2,2,1,2,2,1,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,10,10,3,3,8,8,3,5,3,2,4,10,4,2,1,2,2,7,5,4,9,7,3,3,...,44,55,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,10,10,7,3,4,6,1,2,3,3,10,1,3,4,2,5,1,2,1,<NA>,1,1,3,1,2,1,5,1,1,1,3,1,3,1,3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,2,<NA>,<NA>,9,3,3,0,0,3,6,4,7,3,10,5,4,2,2,0,1,0,0,0,0,0,0,0,0,0,<NA>,2,<NA>,1,10,<NA>,0,0,0,0,0,0,0,1,0,0,0,36,1,<NA>
2,BG,0.315025,0.113172,0.217716,0.024639,0.001932,390,5,405,5,3,4,3,3,2,2,2,3,3,3,3,2,6,5,6,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2,2,2,2,2,2,<NA>,2,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,4,5,5,2,2,3,2,4,4,3,3,6,1,2,1,3,2,6,7,6,5,4,4,5,...,5,55,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,6,5,4,2,2,6,3,3,3,3,5,3,4,1,4,5,3,3,4,4,5,2,3,4,<NA>,2,4,4,2,2,3,4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,5,5,8,4,2,2,<NA>,<NA>,8,5,6,6,3,3,4,3,3,3,5,3,4,2,4,0,0,0,1,0,0,0,0,0,0,0,<NA>,2,<NA>,1,7,<NA>,0,0,0,0,0,0,0,1,0,0,0,93,1,<NA>
3,BG,0.673037,1.436375,0.217716,0.312722,0.000904,60,5,80,5,5,3,4,1,1,1,<NA>,2,2,3,0,0,3,3,3,2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2,2,2,2,2,2,<NA>,2,2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,4,2,2,0,1,1,2,3,2,4,4,3,3,1,3,2,<NA>,<NA>,5,3,4,1,3,...,<NA>,10,2,1,1,7,7,7,10,

In [66]:
thresh = 0.5

nulls_percent_df = pd.DataFrame(ess.isna().sum()/len(ess)).reset_index()
nulls_percent_df.columns = ['column_name', 'nulls_percentage']
display(nulls_percent_df.T.reset_index(drop=True))
cols_under_threshold = list(nulls_percent_df[nulls_percent_df['nulls_percentage']<thresh]['column_name'])

ess[cols_under_threshold]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558
0,cntry,dweight,pspwght,pweight,anweight,prob,nwspol,netusoft,netustm,ppltrst,pplfair,pplhlp,polintr,psppsgva,actrolga,psppipla,cptppola,trstprl,trstlgl,trstplc,trstplt,trstprt,trstep,trstun,trstsci,vote,prtvtebg,prtvthch,prtvtbhr,prtvtecz,prtvthee,prtvtefi,prtvtefr,prtvtdgr,prtvtghu,prtvtdis,prtvtdit,prtvclt1,prtvclt2,prtvclt3,prtvtame,prtvthnl,prtvtmk,prtvtbno,prtvtdpt,prtvtfsi,prtvtesk,contplt,donprty,badge,sgnptit,pbldmna,bctprd,pstplonl,volunfp,clsprty,prtclebg,prtclhch,prtclbhr,prtclecz,prtclhee,prtclffi,prtclffr,prtcldgr,prtclhhu,prtcldis,prtcleit,prtclclt,prtclame,prtclgnl,prtclmk,prtclbno,prtclfpt,prtclfsi,prtclesk,prtdgcl,lrscale,stflife,stfeco,stfgov,stfdem,stfedu,stfhlth,gincdif,freehms,hmsfmlsh,hmsacld,euftf,lrnobed,loylead,imsmetn,imdfetn,impcntr,imbgeco,imueclt,imwbcnt,happy,sclmeet,inprdsc,sclact,crmvct,aesfdrk,health,hlthhmp,atchctr,atcherp,rlgblg,rlgdnm,rlgdnach,rlgdnafi,rlgdnagr,rlgdnhu,rlgdnais,rlgdnlt,rlgdme,rlgdnanl,rlgdnmk,rlgdnno,rlgdnbsk,rlgblge,rlgdnme,rlgdeach,rlgdeafi,rlgdeagr,rlgdehu,rlgdeais,rlgdelt,rlgdeme,rlgdeanl,rlgdemk,rlgdeno,rlgdebsk,rlgdgr,rlgatnd,pray,dscrgrp,dscrrce,dscrntn,dscrrlg,dscrlng,dscretn,dscrage,dscrgnd,dscrsex,dscrdsb,dscroth,dscrdk,dscrref,dscrnap,dscrna,ctzcntr,brncntr,livecnta,feethngr,facntr,mocntr,ccnthum,ccrdprs,wrclmch,admrclc,testic34,testic35,testic36,testic37,testic38,testic39,testic40,testic41,testic42,vteurmmb,vteubcmb,fairelc,dfprtal,medcrgv,rghmgpr,votedir,cttresa,gptpelc,gvctzpv,grdfinc,viepol,wpestop,keydec,fairelcc,dfprtalc,medcrgvc,rghmgprc,votedirc,cttresac,gptpelcc,gvctzpvc,grdfincc,viepolc,wpestopc,keydecc,chpldm,chpldmi,chpldmc,stpldmi,stpldmc,admit,showcv,impdema,impdemb,impdemc,impdemd,impdeme,implvdm,accalaw,hhmmb,gndr,gndr2,gndr3,gndr4,gndr5,gndr6,gndr7,gndr8,gndr9,gndr10,gndr11,gndr12,gndr13,yrbrn,agea,yrbrn2,yrbrn3,yrbrn4,yrbrn5,yrbrn6,yrbrn7,yrbrn8,yrbrn9,yrbrn10

,cntry,dweight,pspwght,pweight,anweight,prob,nwspol,netusoft,netustm,ppltrst,pplfair,pplhlp,polintr,psppsgva,actrolga,psppipla,cptppola,trstprl,trstlgl,trstplc,trstplt,trstprt,trstep,trstun,trstsci,vote,contplt,donprty,badge,sgnptit,pbldmna,bctprd,pstplonl,volunfp,clsprty,lrscale,stflife,stfeco,stfgov,stfdem,stfedu,stfhlth,gincdif,freehms,hmsfmlsh,hmsacld,euftf,lrnobed,loylead,imsmetn,imdfetn,impcntr,imbgeco,imueclt,imwbcnt,happy,sclmeet,inprdsc,sclact,crmvct,aesfdrk,health,hlthhmp,atchctr,atcherp,rlgblg,rlgdnm,rlgdgr,rlgatnd,pray,dscrgrp,dscrrce,dscrntn,dscrrlg,dscrlng,dscretn,dscrage,dscrgnd,dscrsex,dscrdsb,dscroth,dscrdk,dscrref,dscrnap,dscrna,ctzcntr,brncntr,feethngr,facntr,mocntr,ccnthum,ccrdprs,wrclmch,admrclc,vteurmmb,fairelc,dfprtal,medcrgv,rghmgpr,votedir,cttresa,gptpelc,gvctzpv,grdfinc,viepol,wpestop,keydec,fairelcc,dfprtalc,medcrgvc,rghmgprc,votedirc,cttresac,gptpelcc,gvctzpvc,grdfincc,viepolc,wpestopc,keydecc,chpldm,chpldmi,chpldmc,admit,showcv,implvdm,accalaw,hhmmb,gndr,gndr2,yrbrn,agea,yrbrn2,rshipa2,rshpsts,lvgptnea,dvrcdeva,marsts,maritalb,chldhhe,domicil,edulvlb,eisced,eduyrs,pdwrk,edctn,uempla,uempli,dsbld,rtrd,cmsrv,hswrk,dngoth,dngref,dngdk,dngna,mnactic,emplrel,wrkctra,estsz,jbspv,wkdcorga,iorgact,wkhct,wkhtot,nacer2,tporgwk,isco08,wrkac6m,uemp3m,mbtru,hincsrca,hinctnta,hincfel,edulvlpb,eiscedp,pdwrkp,edctnp,uemplap,uemplip,dsbldp,rtrdp,cmsrvp,hswrkp,dngothp,dngdkp,dngnapp,dngrefp,dngnap,edulvlfb,eiscedf,emprf14,occf14b,edulvlmb,eiscedm,emprm14,occm14b,atncrse,anctry1,anctry2,regunit,region,acchome,accwrk,accmove,accoth,accnone,accref,accdk,accna,fampref,famadvs,fampdf,mcclose,mcinter,mccoord,mcpriv,mcmsinf,chldo12,ageo12,hhlio12,speako12,scrno12,phoneo12,como12,c19spo12,c19mco12,livpnt,agepnt,hhlipnt,closepnt,speakpnt,scrnpnt,phonepnt,compnt,c19sppnt,c19mcpnt,stfmjob,trdawrk,jbprtfp,pfmfdjba,dcsfwrka,wrkhome,c19whome,wrklong,mansupp,teamfeel,colprop,ipcrtiv,imprich,ipeqopt,ipshabt,impsafe,impdiff,ipfrule,ipudrst,ipmodst,ipgdtim,impfree,iphlppl,ipsuces,ipstrgv,ipadvnt,ipbhprp,iprspot,iplylfr,impenv,imptrad,impfun,secgrdec,scidecpb,admc19,panfolru,panclobo,panresmo,gvhanc19,gvjobc19,gveldc19,gvfamc19,hscopc19,gvbalc19,gvimpc19,gvconc19,respc19,reshhc19,hapljc19,hapirc19,hapwrc19,hapfuc19,hapfoc19,hapnoc19,hapnwc19,hapnpc19,haprec19,hapdkc19,hapnac19,getavc19,vdcond,vdovexre,vdtpsvre,vdtpitre,vdtpscre,vdtpaure,vdtpvire,vdtpoire,vdtpntre,vdtpapre,vdtprere,vdtpdkre,vdtpnare,inwtm,mode
0,BG,1.939384,1.290706,0.217716,0.281008,0.000314,80,1,<NA>,5,5,1,4,1,2,2,2,3,2,3,3,3,4,4,6,2,2,2,2,2,2,2,2,2,2,<NA>,8,3,4,4,3,4,4,3,3,3,2,3,3,3,3,3,2,2,2,6,6,2,3,2,2,2,3,8,6,1,3,2,6,6,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,1,5,7,4,1,33,9,8,7,8,8,8,8,8,7,7,7,8,1,2,1,9,2,1,1,2,1,2,1,2,1,9,1,3,1,3,1,2,2,1,1945,76,1941,1,1,2,2,<NA>,1,1,1,323,4,12,0,0,0,0,0,1,0,0,0,0,0,0,6,1,1,4,2,3,2,40,40,14,3,7531,2,2,2,4,7,2,323,4,0,0,0,0,0,1,0,0,0,0,0,0,0,113,1,1,9,113,1,1,9,2,14030,555555,3,BG331,0,0,0,0,1,0,0,0,1,1,1,<NA>,<NA>,<NA>,<NA>,<NA>,1,52,2,3,7,4,7,3,3,4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2,2,2,5,3,1,2,4,4,2,1,3,5,3,3,2,5,4,3,1,5,2,1,2,8,3,8,7,1,8,8,8,5,8,1,3,4,0,0,0,0,0,0,0,1,0,0,0,3,1,9,0,0,0,0,0,0,0,1,0,0,0,53,1
1,BG,1.651595,1.430878,0.217716,0.311526,0.000368,63,5,180,0,6,2,1,4,4,4,4,5,8,9,6,7,8,8,10,1,1,2,2,2,2,1,2,2,1,10,10,3,3,8,8,3,5,3,2,4,10,4,2,1,2,2,7,5,4,9,7,3,3,2,1,2,3,10,8,1,3,6,6,6,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,1,5,8,3,1,1,8,10,9,0,10,10,8,2,7,6,3,<NA>,10,0,9,10,0,4,10,6,3,4,3,0,5,<NA>,8,2,1,10,0,4,1,2,1978,43,1978,1,1,1,2,<NA>,1,<NA>,4,620,6,16,1,0,0,0,0,0,0,0,0,0,0,0,1,3,1,1,1,10,10,40,56,49,5,1120,2,1,3,2,<NA>,1,720,7,1,0,0,0,0,0,0,0,0,0,0,0,0,323,4,1,4,313,4,1,3,2,14030,555555,3,BG334,1,1,1,1,0,0,0,0,4,5,5,10,0,10,7,7,1,12,1,1,6,1,4,3,3,1,75,2,2,3,7,3,5,3,3,9,4,3,3,3,1,2,6,55,10,7,1,3,4,2,5,1,2,1,<NA>,1,1,3,1,2,1,5,1,1,1,3,1,1,<NA>,2,3,0,0,3,6,4,7,3,10,5,4,2,2,0,1,0,0,0,0,0,0,0,0,0,2,1,10,0,0,0,0,0,0,0,1,0,0,0,36,1
2,BG,0.315025,0.113172,0.21771

In [70]:
# in the following i will filter columns i want to inspect in my project
# this selection can be made very differently, but the data is now prepared for it

features = [
    'cntry', 'nwspol', 'polintr', 'sgnptit', 'pbldmna', 'happy', 'sclmeet',     # age, gender, religion, employment, health, wealth
    'inprdsc', 'health', 'domicil', 'edulvlb', 'eisced', 'rlgblg',
    'rlgdgr', 'medcrgv', 'gndr', 'agea', 'mainact', 'mnactic', 'hincfel'
] + [t for t in ess.columns if t.startswith('net')      # use of internet
] + [t for t in ess.columns if t.startswith('trst')     # trust
] + [t for t in ess.columns if t.startswith('ppl')      # people
] + [t for t in ess.columns if t.startswith('stf')      # satisfaction
] + [t for t in ess.columns if t.startswith('acc')      # access to internet
] + [t for t in ess.columns if t.startswith('atch')     # emotionally attached
] + [t for t in ess.columns if t.startswith('mc')       # online/mobile communication
]

thresh = 0.5

nulls_percent_df = pd.DataFrame(ess[features].isna().sum()/len(ess[features])).reset_index()
nulls_percent_df.columns = ['column_name', 'nulls_percentage']
display(nulls_percent_df.T.reset_index(drop=True))
cols_under_threshold = list(nulls_percent_df[nulls_percent_df['nulls_percentage']<thresh]['column_name'])

ess[cols_under_threshold]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56
0,cntry,nwspol,polintr,sgnptit,pbldmna,happy,sclmeet,inprdsc,health,domicil,edulvlb,eisced,rlgblg,rlgdgr,medcrgv,gndr,agea,mainact,mnactic,hincfel,netusoft,netustm,trstprl,trstlgl,trstplc,trstplt,trstprt,trstep,trstun,trstsci,ppltrst,pplfair,pplhlp,stflife,stfeco,stfgov,stfdem,stfedu,stfhlth,stfmjob,accalaw,acchome,accwrk,accmove,accoth,accnone,accref,accdk,accna,atchctr,atcherp,mcclose,mcinter,mccoord,mcpriv,mcmsinf,mcwrkhom
1,0.0,0.017751,0.002459,0.008695,0.008545,0.002459,0.004198,0.015562,0.001259,0.001439,0.003748,0.003748,0.008575,0.011514,0.01844,0.0,0.007916,0.903481,0.004678,0.013493,0.002039,0.274505,0.017541,0.016551,0.007286,0.012473,0.015892,0.059488,0.07466,0.233516,0.003328,0.005847,0.003748,0.010285,0.020839,0.023538,0.023178,0.049924,0.008246,0.449222,0.03793,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005967,0.012383,0.03721,0.134239,0.048364,0.047195,0.077659,0.585889


,cntry,nwspol,polintr,sgnptit,pbldmna,happy,sclmeet,inprdsc,health,domicil,edulvlb,eisced,rlgblg,rlgdgr,medcrgv,gndr,agea,mnactic,hincfel,netusoft,netustm,trstprl,trstlgl,trstplc,trstplt,trstprt,trstep,trstun,trstsci,ppltrst,pplfair,pplhlp,stflife,stfeco,stfgov,stfdem,stfedu,stfhlth,stfmjob,accalaw,acchome,accwrk,accmove,accoth,accnone,accref,accdk,accna,atchctr,atcherp,mcclose,mcinter,mccoord,mcpriv,mcmsinf
0,BG,80,4,2,2,6,6,2,2,1,323,4,1,2,7,2,76,6,2,1,<NA>,3,2,3,3,3,4,4,6,5,5,1,8,3,4,4,3,4,<NA>,1,0,0,0,0,1,0,0,0,8,6,<NA>,<NA>,<NA>,<NA>,<NA>
1,BG,63,1,2,2,9,7,3,2,4,620,6,1,6,9,1,43,1,1,5,180,5,8,9,6,7,8,8,10,0,6,2,10,3,3,8,8,3,9,0,1,1,1,1,0,0,0,0,10,8,10,0,10,7,7
2,BG,390,3,2,2,5,4,4,3,1,720,7,1,5,8,2,50,1,2,5,405,3,3,3,3,2,6,5,6,5,3,4,5,5,2,2,3,2,5,2,0,1,0,0,0,0,0,0,8,8,2,7,5,7,7
3,BG,60,4,2,2,3,4,1,3,4,313,4,1,6,10,2,51,1,3,5,80,2,2,3,0,0,3,3,3,5,5,3,4,2,2,0,1,1,7,3,1,0,1,0,0,0,0,0,5,3,6,5,10,0,5
4,BG,120,1,2,1,3,4,0,2,4,720,7,1,4,10,1,70,6,3,5,120,0,0,0,0,0,0,0,3,4,4,2,3,2,9,3,2,1,4,0,1,1,1,1,0,0,0,0,8,0,5,5,9,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33346,SK,120,4,2,2,5,3,1,3,3,321,3,1,8,10,1,67,6,2,2,<NA>,3,2,9,7,8,4,8,10,5,5,5,8,5,3,4,<NA>,10,<NA>,10,1,0,0,0,0,0,0,0,10,8,2,2,1,7,1
33347,SK,90,2,2,2,8,7,1,2,4,323,4,2,1,10,1,63,6,2,4,120,7,7,8,8,8,8,<NA>,10,7,6,6,9,8,8,10,6,8,<NA>,0,1,0,1,1,0,0,0,0,10,9,10,0,10,0,0
33348,SK,30,2,2,2,7,3,3,2,3,720,7,1,8,5,1,48,1,2,2,<NA>,1,1,0,1,0,4,4,4,0,0,2,7,1,4,7,5,4,9,8,1,0,0,1,0,0,0,0,9,7,8,8,8,6,6
33349,SK,30,4,2,2,7,6,1,2,4,720,7,1,3,9,2,49,1,2,5,90,2,0,8,7,3,4,5,9,2,2,3,2,3,2,2,7,7,9,7,1,1,1,1,0,0,0,0,6,7,9,3,10,3,5
